In [1]:
%pip install mglearn
%pip install cvxpy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
#Libraries
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as la
import pandas as pd
import cvxpy as cp
import mglearn
import math
from sklearn.preprocessing import MinMaxScaler

In [3]:
#read data into a var and print column names
enc_data = pd.read_csv('traffic_encryption_data.csv')
print(enc_data.columns)

Index(['packet_size', 'transmission_time', 'latency', 'jitter', 'packet_loss',
       'network_load', 'throughput', 'bandwidth_usage', 'encryption_time',
       'decryption_time', 'key_size', 'block_size', 'cpu_usage',
       'memory_usage', 'encryption_strength', 'error_rate',
       'integrity_check_time', 'retransmission_count', 'packet_order',
       'traffic_type'],
      dtype='object')


In [4]:
#selected columns coppied
selected_cols = ['encryption_time', 'encryption_strength', 'error_rate', 'integrity_check_time', 'cpu_usage',
       'memory_usage']
clean_enc_data = enc_data[selected_cols]
print(clean_enc_data)

      encryption_time  encryption_strength  error_rate  integrity_check_time  \
0            7.247682                    5    1.021120              0.405453   
1            4.867058                    7    0.026064              0.878333   
2            3.618133                    8    0.361215              0.949209   
3            5.432145                    4    0.995241              1.711475   
4            4.559455                    2    1.505134              4.173365   
...               ...                  ...         ...                   ...   
2995         8.571986                    3    0.966220              3.715642   
2996         9.653815                    7    1.311826              4.499620   
2997         7.007885                   10    1.922128              1.705988   
2998         3.612489                    4    0.050092              4.889710   
2999         3.431766                    1    1.352565              2.066367   

      cpu_usage  memory_usage  
0     5

In [5]:
#may need to normalize this apparently does it 
scaler = MinMaxScaler()
normalized = pd.DataFrame(scaler.fit_transform(clean_enc_data), columns=selected_cols)

#define varaibles
#normalized
enc_times = normalized['encryption_time'].values
enc_strengths = normalized['encryption_strength'].values
err_rates = normalized['error_rate'].values
integ_times = normalized['integrity_check_time'].values
cpu_usages = normalized['cpu_usage'].values
mem_usages = normalized['memory_usage'].values
#not normalized
'''
enc_times = clean_enc_data['encryption_time'].copy()
enc_strengths = clean_enc_data['encryption_strength'].copy()
err_rates = clean_enc_data['error_rate'].copy()
integ_times = clean_enc_data['integrity_check_time'].copy()
cpu_usages = clean_enc_data['cpu_usage'].copy()
mem_usages = clean_enc_data['memory_usage'].copy()
'''


"\nenc_times = clean_enc_data['encryption_time'].copy()\nenc_strengths = clean_enc_data['encryption_strength'].copy()\nerr_rates = clean_enc_data['error_rate'].copy()\ninteg_times = clean_enc_data['integrity_check_time'].copy()\ncpu_usages = clean_enc_data['cpu_usage'].copy()\nmem_usages = clean_enc_data['memory_usage'].copy()\n"

In [6]:
# Grid search parameters playing with to see how results change
alphas = [0.1, 0.5, 1.5]
lambdas = [0.1, 1.0, 10.0]
gammas = [0.01, 0.1, 1.0]
results = []

num_scenarios = len(enc_times)

for lam_1 in lambdas:
    for lam_2 in lambdas:
        for lam_3 in lambdas:
            for gamma in gammas:
                for alpha in alphas:
                    try:
                        # NEW: define w inside the loop
                        w = cp.Variable(num_scenarios)

                        # Objective components
                        Q_diag = enc_times + lam_1 * (1 - enc_strengths) + lam_2 * err_rates - lam_3 * integ_times
                        Q_diag = np.maximum(Q_diag, 0)
                        Q_matrix = np.diag(Q_diag)
                        sec_perf_tradeoff = cp.quad_form(w, Q_matrix)

                        c = cpu_usages + mem_usages
                        resource_term = gamma * c @ w

                        # L1 regularization for sparsity
                        l1_term = alpha * cp.norm1(w)

                        # Full objective
                        obj = cp.Minimize(sec_perf_tradeoff + resource_term + l1_term)

                        # Constraints
                        constraints = [
                            cp.sum(w) == 1,
                            w >= 0,
                            enc_strengths @ w >= 0.3
                        ]

                        # Solve
                        prob = cp.Problem(obj, constraints)
                        sol = prob.solve(solver=cp.SCS)

                        if sol is not None and np.isfinite(sol):
                            w_clean = np.clip(w.value, 0, 1)
                            chosen_indexes = np.where(w_clean > 1e-4)[0]
                            chosen_weights = w_clean[chosen_indexes]


                            results.append({
                                'lam_1': lam_1,
                                'lam_2': lam_2,
                                'lam_3': lam_3,
                                'gamma': gamma,
                                'alpha': alpha,
                                'min_cost': sol,
                                'nonzero_configs': len(chosen_indexes),
                                'chosen_configs': chosen_indexes.tolist(),
                                'chosen_weights': chosen_weights.tolist()
                            })

                    except Exception as e:
                        print(f"Skipped combo lam1={lam_1}, alpha={alpha}: {e}")
                        continue

# Convert results to DataFrame
results_df = pd.DataFrame(results)
print(results_df.head())

   lam_1  lam_2  lam_3  gamma  alpha  min_cost  nonzero_configs  \
0    0.1    0.1    0.1   0.01    0.1  0.101805               61   
1    0.1    0.1    0.1   0.01    0.5  0.501798               61   
2    0.1    0.1    0.1   0.01    1.5  1.501914               61   
3    0.1    0.1    0.1   0.10    0.1  0.109675               30   
4    0.1    0.1    0.1   0.10    0.5  0.509089               30   

                                      chosen_configs  \
0  [25, 49, 162, 237, 405, 415, 544, 643, 656, 66...   
1  [25, 49, 162, 237, 405, 415, 544, 643, 656, 66...   
2  [25, 49, 162, 237, 405, 415, 544, 643, 656, 66...   
3  [49, 405, 415, 544, 656, 663, 951, 1105, 1389,...   
4  [49, 405, 415, 544, 656, 663, 951, 1105, 1389,...   

                                      chosen_weights  
0  [0.00036845727135078606, 0.001854457453696649,...  
1  [0.00038164913436785085, 0.0018613011635856221...  
2  [0.0003601916167985284, 0.0018501718922842583,...  
3  [0.009433293512383631, 0.009015901755

In [7]:
# Get the best result (lowest cost)
best_row = results_df.sort_values("min_cost").iloc[0]
indexes = best_row["chosen_configs"]
weights = best_row["chosen_weights"]
min_cost = best_row["min_cost"]

# Print min cost
print(f"\n🧠 Best Configuration Set — Min Cost: {min_cost:.8f}\n")

# Print table: index, weight, time, strength, key size
print(f"{'Config Index':<15} {'Weight (w)':>12} {'Enc. Time':>12} {'Strength':>10} {'Key Size':>10}")
print("-" * 65)
for idx, w_val in zip(indexes, weights):
    row = clean_enc_data.iloc[idx]
    enc_time = row["encryption_time"]
    strength = row["encryption_strength"]
    key_size = enc_data.iloc[idx]["key_size"]  # key_size was in original dataset
    print(f"{idx:<15} {w_val:>12.6f} {enc_time:>12.6f} {strength:>10.2f} {key_size:>10}")

# Optional: full row dump
print("\n📋 Full Rows from Original Data (Top Configs):")
chosen_configs = clean_enc_data.iloc[indexes]
print(chosen_configs.reset_index(drop=True))



🧠 Best Configuration Set — Min Cost: 0.10007712

Config Index      Weight (w)    Enc. Time   Strength   Key Size
-----------------------------------------------------------------
1661                0.999291     1.079342       9.00      256.0

📋 Full Rows from Original Data (Top Configs):
   encryption_time  encryption_strength  error_rate  integrity_check_time  \
0         1.079342                    9    0.214857              3.487622   

   cpu_usage  memory_usage  
0  10.210532    102.027749  


In [8]:
# Extract summary from the results
summary = results_df[['lam_1', 'lam_2', 'lam_3', 'gamma', 'alpha', 'min_cost', 'chosen_configs', 'chosen_weights', 'nonzero_configs']].copy()

# Get top 3 (config index, weight) pairs
summary['top_3_configs'] = summary.apply(
    lambda row: list(zip(row['chosen_configs'][:3], row['chosen_weights'][:3])),
    axis=1
)

# Print first 10 rows of summary
print(summary[['lam_1', 'lam_2', 'lam_3', 'gamma', 'alpha', 'min_cost', 'nonzero_configs', 'top_3_configs']])


     lam_1  lam_2  lam_3  gamma  alpha  min_cost  nonzero_configs  \
0      0.1    0.1    0.1   0.01    0.1  0.101805               61   
1      0.1    0.1    0.1   0.01    0.5  0.501798               61   
2      0.1    0.1    0.1   0.01    1.5  1.501914               61   
3      0.1    0.1    0.1   0.10    0.1  0.109675               30   
4      0.1    0.1    0.1   0.10    0.5  0.509089               30   
..     ...    ...    ...    ...    ...       ...              ...   
238   10.0   10.0   10.0   0.10    0.5  0.500522                1   
239   10.0   10.0   10.0   0.10    1.5  1.500503                1   
240   10.0   10.0   10.0   1.00    0.1  0.104701                2   
241   10.0   10.0   10.0   1.00    0.5  0.504722                2   
242   10.0   10.0   10.0   1.00    1.5  1.504701                2   

                                         top_3_configs  
0    [(25, 0.00036845727135078606), (49, 0.00185445...  
1    [(25, 0.00038164913436785085), (49, 0.00186130...  


In [9]:
summary.to_csv("summary_hyperparam_results.csv", index=False)